### 案例1 MNIST-手写数字识别系统
+ 算法思路
    > 准备训练参数

    > 准备数据集(训练集和测试集)

    > 建立神经网络模型

    > 选择优化算法, 计算 loss
    
    > 训练集上训练模型
    
    > 测试集上检验正确率

In [1]:
# 模块导入
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters 
input_size = 784 # 输入
num_classes = 10
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)

# train_dataset

train_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

# test_dataset
test_dataset = torchvision.datasets.MNIST(root='./data/',
                                         train=False,
                                         transform=transforms.ToTensor())

In [4]:
# data loader (input pipeline)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,   # 导出图片数据集
                                           batch_size=batch_size,   # 训练选取多少张图片
                                           shuffle=True)            # 采取多少个子进程

In [5]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [6]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.lr = nn.Linear(784, 10)
        self.sm = nn.Sigmoid()
        
    def forward(self, x):
        x = self.lr(x)
        x = self.sm(x)
        
        return x
    

In [7]:
logistic_model = LogisticRegression()
if torch.cuda.is_available():
    logistic_model.cuda()
    

In [9]:
# loss and optimizer
# nn.CrossEntropyLoss() computes sfotmax internally

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(logistic_model.parameters(),lr=learning_rate)

In [10]:
# Train the model

total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1,28*28)
        
        # Forward pass
        outputs = logistic_model(images)
        loss = criterion(outputs,labels)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/50], Step [100/600], Loss: 2.2970
Epoch [1/50], Step [200/600], Loss: 2.2959
Epoch [1/50], Step [300/600], Loss: 2.2860
Epoch [1/50], Step [400/600], Loss: 2.2793
Epoch [1/50], Step [500/600], Loss: 2.2732
Epoch [1/50], Step [600/600], Loss: 2.2674
Epoch [2/50], Step [100/600], Loss: 2.2602
Epoch [2/50], Step [200/600], Loss: 2.2520
Epoch [2/50], Step [300/600], Loss: 2.2556
Epoch [2/50], Step [400/600], Loss: 2.2451
Epoch [2/50], Step [500/600], Loss: 2.2314
Epoch [2/50], Step [600/600], Loss: 2.2235
Epoch [3/50], Step [100/600], Loss: 2.2229
Epoch [3/50], Step [200/600], Loss: 2.2128
Epoch [3/50], Step [300/600], Loss: 2.2185
Epoch [3/50], Step [400/600], Loss: 2.2097
Epoch [3/50], Step [500/600], Loss: 2.2022
Epoch [3/50], Step [600/600], Loss: 2.1941
Epoch [4/50], Step [100/600], Loss: 2.1994
Epoch [4/50], Step [200/600], Loss: 2.1823
Epoch [4/50], Step [300/600], Loss: 2.1779
Epoch [4/50], Step [400/600], Loss: 2.1683
Epoch [4/50], Step [500/600], Loss: 2.1702
Epoch [4/50

Epoch [32/50], Step [300/600], Loss: 1.8507
Epoch [32/50], Step [400/600], Loss: 1.8417
Epoch [32/50], Step [500/600], Loss: 1.8356
Epoch [32/50], Step [600/600], Loss: 1.8727
Epoch [33/50], Step [100/600], Loss: 1.8510
Epoch [33/50], Step [200/600], Loss: 1.8542
Epoch [33/50], Step [300/600], Loss: 1.8519
Epoch [33/50], Step [400/600], Loss: 1.8253
Epoch [33/50], Step [500/600], Loss: 1.8227
Epoch [33/50], Step [600/600], Loss: 1.8354
Epoch [34/50], Step [100/600], Loss: 1.7945
Epoch [34/50], Step [200/600], Loss: 1.8284
Epoch [34/50], Step [300/600], Loss: 1.8273
Epoch [34/50], Step [400/600], Loss: 1.8541
Epoch [34/50], Step [500/600], Loss: 1.8337
Epoch [34/50], Step [600/600], Loss: 1.8700
Epoch [35/50], Step [100/600], Loss: 1.8263
Epoch [35/50], Step [200/600], Loss: 1.8734
Epoch [35/50], Step [300/600], Loss: 1.8539
Epoch [35/50], Step [400/600], Loss: 1.8303
Epoch [35/50], Step [500/600], Loss: 1.8327
Epoch [35/50], Step [600/600], Loss: 1.8828
Epoch [36/50], Step [100/600], L

In [12]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = logistic_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(logistic_model.state_dict(), 'model.ckpt')

Accuracy of the model on the 10000 test images: 84 %
